In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import pandas as pd

from kiwipiepy import Kiwi

from collections import Counter

import time

In [3]:
kiwi = Kiwi()

def count_word(text, word_counts):
    # kiwi.analyze()로 분석 후, 품사가 'NNG'(일반 명사), 'NNP'(고유 명사)인 경우만 추출
    nouns = [token.form for token in kiwi.analyze(text)[0][0] if token.tag.startswith('NN')]
    new_word_counts = Counter(nouns) + word_counts
    return new_word_counts

In [5]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))

# url에 포함되어 있는 parameters
# 정렬 : sort={"0":관련도순, "1":최신순, "2":오래된순}
# 기간 : (시작기간)ds=YYYY.MM.DD, (끝기간)de=YYYY.MM.DD
# 유형 : photo={"0":전체, "1":포토, "2":동영상, "3":지면기사, ...}
# mynews=1, office_type=3, office_category=3으로 수정
company_name = "폭락"  # 검색어
date = "2024.08.05"      # 검색기간(하루로 제한 -> 추후 수정 예정)
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&query={company_name}&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds={date}&de={date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date.replace(".","")}to{date.replace(".","")}&is_sug_officeid=0&office_category=3&service_area=0"
driver.get(url)
time.sleep(2)            # 사이트 차단 방지를 위한 멈춤

In [ ]:
# 기간 내 기사들의 링크 모두 가져오기

# 모든 기사가 보일때까지 스크롤
len_before_scroll = 0
while (True):
    # 현재 로드된 뉴스 기사 목록을 모두 가져온다 (CSS 선택자는 실제 페이지에 맞게 수정)
    articles = driver.find_elements(By.CSS_SELECTOR, "div.BHQHyn3Flk5rFBSacJkG")

    # 스크롤 후 기사 수
    len_after_scroll = len(articles)

    if len(articles) > 100:
        break

    # 목록의 끝에서 두 번째 항목을 타겟으로 지정 (마지막 항목은 푸터와 겹칠 수 있음)
    if len(articles) > 1:
        target_element = articles[-2]
        
        # JavaScript를 실행하여 해당 요소가 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView()", target_element)

        time.sleep(1)

    # 스크롤 전후 기사 수 비교 (같다면 더이상 스크롤 할 필요 X)
    if len_before_scroll == len_after_scroll:
        break

    len_before_scroll = len_after_scroll

In [14]:
# 네이버 뉴스 주소 가져오기

news_urls = []
news_info = driver.find_elements(By.CLASS_NAME, "sds-comps-horizontal-layout.sds-comps-full-layout.sds-comps-profile.type-basic.size-lg.title-color-g10.hRlTUxpL38wmDKxyAhSU")
for info in news_info:
    news_link = info.find_element(By.CLASS_NAME, "GUWgsNcVrWa67MoYor6N.xR1x3GgF_MIcYPUZNqEu")
    news_urls.append(news_link.get_attribute("href"))

In [ ]:
driver.quit()

In [33]:
def get_news_content(url):
    req = Request(url, headers={'User-Agent': generate_user_agent()})
    page = urlopen(req)
    return BeautifulSoup(page, "html.parser")

In [ ]:

word_counts=Counter()
for url in news_urls:
    soup = get_news_content(url)
    time.sleep(1)
    news_title = soup.find("div", class_="media_end_head_title").find("h2")
    news_date = soup.find("span", class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME")['data-date-time']
    # print(news_date.split(" ")[0])
    news_content = soup.find("article", class_="go_trans _article_content")

    word_counts = count_word(news_content.text, word_counts)

print(word_counts)

Counter({'시장': 196, '것': 191, '증시': 183, '미국': 177, '월': 173, '일': 167, '지수': 165, '하락': 162, '경기': 154, '침체': 148, '등': 142, '년': 139, '경제': 133, '이날': 117, '원': 115, '코스피': 108, '폭락': 106, '수': 102, '금리': 97, '우려': 96, '기록': 86, '이후': 81, '만': 72, '코스닥': 69, '주가': 60, '대비': 59, '상황': 59, '인하': 58, '공포': 57, '최대': 56, '장': 55, '금융': 55, '정부': 54, '거래': 54, '지표': 53, '법': 52, '일본': 52, '국내': 51, '개월': 50, '글로벌': 50, '포인트': 49, '역대': 49, '기준': 47, '낙폭': 46, '발동': 45, '가능': 45, '폭': 44, '주': 44, '거래일': 44, '자산': 43, '투자': 42, '중동': 42, '급락': 41, '달러': 41, '개': 39, '연준': 39, '전망': 39, '수준': 39, '중': 39, '위기': 39, '시': 38, '아시아': 38, '선': 37, '서킷브레이커': 37, '말': 37, '한국': 36, '분석': 36, '법안': 34, '때': 34, '종목': 34, '세계': 33, '고용': 32, '투자자': 32, '마감': 32, '국회': 31, '중국': 31, '대응': 31, '대만': 31, '저가': 31, '올해': 30, '실업': 30, '예상': 30, '기업': 29, '대': 29, '주식': 29, '시간': 29, '심리': 29, '정책': 28, '반도체': 28, '영향': 27, '엔': 27, '매도': 27, '대통령': 26, '닛케이': 26, '발표': 26, '이상': 26, '상승': 26, '자금': 26,

In [ ]:
forbidden_words = ["등", "것", "년","월", "일","이날"]

for word in list(word_counts.keys()):
    if word in forbidden_words:
        del word_counts[word]

In [50]:
word_counts

Counter({'시장': 196,
         '증시': 183,
         '미국': 177,
         '지수': 165,
         '하락': 162,
         '경기': 154,
         '침체': 148,
         '경제': 133,
         '이날': 117,
         '원': 115,
         '코스피': 108,
         '폭락': 106,
         '수': 102,
         '금리': 97,
         '우려': 96,
         '기록': 86,
         '이후': 81,
         '만': 72,
         '코스닥': 69,
         '주가': 60,
         '대비': 59,
         '상황': 59,
         '인하': 58,
         '공포': 57,
         '최대': 56,
         '장': 55,
         '금융': 55,
         '정부': 54,
         '거래': 54,
         '지표': 53,
         '법': 52,
         '일본': 52,
         '국내': 51,
         '개월': 50,
         '글로벌': 50,
         '포인트': 49,
         '역대': 49,
         '기준': 47,
         '낙폭': 46,
         '발동': 45,
         '가능': 45,
         '폭': 44,
         '주': 44,
         '거래일': 44,
         '자산': 43,
         '투자': 42,
         '중동': 42,
         '급락': 41,
         '달러': 41,
         '개': 39,
         '연준': 39,
         '전망': 39,
  